### 환경 install

In [1]:
#pip install langchain-google-genai


In [2]:
#pip install langchain langchain-community langchain-core langchain-text-splitters langchain-google-genai faiss-cpu pymupdf

In [3]:
#pip install langchain-postgres psycopg2-binary

In [4]:
#pip install google-generativeai

In [5]:
# pip install langchain langchain-community langchain-openai

In [6]:
# pip install langchain-openai

In [7]:
# pip install lark

In [8]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv
load_dotenv()

True

In [9]:
# from langchain_text_splitters import RecursiveCharacterTextSplitter
# from langchain_community.document_loaders import PyMuPDFLoader
# from langchain_community.vectorstores import FAISS
# from langchain_core.output_parsers import StrOutputParser
# from langchain_core.runnables import RunnablePassthrough
# from langchain_core.prompts import PromptTemplate
# from langchain_community.document_loaders import TextLoader
# from langchain_community.vectorstores import PGVector
# # from langchain_community.retrievers import MergerRetriever  

# # 변경 부분 - Google genai, and PGVector
# from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
# from langchain_postgres.vectorstores import PGVector
# from langchain_community.document_loaders import DirectoryLoader, TextLoader
# import time

import os
import pandas as pd
import numpy as np
import openai
from datetime import datetime
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import PGVector
from dotenv import load_dotenv
# from langchain_community.retrievers import MergerRetriever  
from langchain_openai import ChatOpenAI
from langchain_classic.retrievers import SelfQueryRetriever

# 변경 부분 - Google genai, and PGVector
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain_postgres.vectorstores import PGVector
from langchain_community.document_loaders import DirectoryLoader, TextLoader
import time
from dotenv import load_dotenv


#Openai 추가
from langchain_openai import OpenAIEmbeddings
from langchain_postgres import PGVector

#셀프쿼리시 추가 library
from langchain_core.structured_query import Comparator, Operator
from langchain_classic.retrievers.self_query.pgvector import PGVectorTranslator
from langchain_classic.retrievers import SelfQueryRetriever
from langchain_classic.chains.query_constructor.schema import AttributeInfo

In [10]:
# API KEY 체크
load_dotenv(override=True)

# API 키 확인
api_key = os.getenv('OPENAI_API_KEY')
print(f"API 키: {api_key[:8]}")
CONNECTION_STRING = os.getenv("DATABASE_URL")
print(f"DB connection string : {CONNECTION_STRING[:3]}")

# 만약 None이면 문제!
if api_key is None:
    print("❌ .env 파일에 OPENAI_API_KEY가 없습니다!")

if CONNECTION_STRING is None:
    print("❌ .env 파일에 CONNECTION_STRING가 없습니다!")

API 키: sk-proj-


## 임베딩 Ai 연결, DB 연결

In [11]:
# 단계 3: 임베딩(Embedding) 생성
#openai 사용 
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# 임베딩 추가 없이 DB만 호출하고 싶을때는 아래 코드 활성화  
vectorstore = PGVector(
    embeddings=embeddings,
    collection_name="SI_try",
    connection=CONNECTION_STRING,  # connection_string이 아니라 connection
)


In [12]:
# # langchain 소스코드 확인하기
# # 흥미로운 내용 많네요 
# ??PGVector


# 일반 유사도 검색(기존)

In [13]:
# 코사인 거리(cosine distance) = 1 - cos(θ)
# - 범위: 0 ~ 2 (보통 0 ~ 1)
# - 0에 가까울수록 유사

#k는 문서 개수, 정의 안하면 기본 4개 
# results = vectorstore.similarity_search_with_score("투썸플레이스 영수증프린터", k=3)
# for i, (doc, score) in enumerate(results, 1):
#     print(f"{i}위 - 유사도: {score:.4f}")
#     print(f"내용: {doc.page_content[:100]}...")
#     print("---")

## 메타 데이터에 대한 데이터 생성 (metadata_field_info)


In [14]:
#embedding 할 때 사용한 metadata 


# def create_metadata(row):
#     """메타데이터 생성"""
#     return {
#         'no': int(row['No']),
#         'date': row['접수일자'].strftime('%Y-%m-%d') if pd.notna(row['접수일자']) else None,
#         'category': str(row['가맹분류']),
#         'store_name': str(row['가맹점명']),
#         'summary': str(row['상담요약']),
#         'type': str(row['처리유형']),
#         'main_category': str(row['대분류']),
#         'sub_category': str(row['중분류']),
#         'year': int(row['접수일자'].year) if pd.notna(row['접수일자']) else None,
#         'month': int(row['접수일자'].month) if pd.notna(row['접수일자']) else None
#     }


In [15]:
metadata_field_info = [
    AttributeInfo(
        name="no",
        description="상담 접수 번호. 고유한 식별자",
        type="integer"
    ),
    AttributeInfo(
        name="date",
        description="상담 접수일자. YYYY-MM-DD 형식의 날짜",
        type="string"
    ),
    AttributeInfo(
        name="category",
        description="가맹점의 분류 카테고리",
        type="string"
    ),
    AttributeInfo(
        name="store_name",
        description="가맹점의 이름",
        type="string"
    ),
    AttributeInfo(
        name="summary",
        description="상담 내용의 요약",
        type="string"
    ),
    AttributeInfo(
        name="type",
        description="상담 처리 유형 (예: 완료, 진행중, 보류 등)",
        type="string"
    ),
    AttributeInfo(
        name="main_category",
        description="상담 내용의 대분류 카테고리",
        type="string"
    ),
    AttributeInfo(
        name="sub_category",
        description="상담 내용의 중분류 카테고리",
        type="string"
    ),
    AttributeInfo(
        name="year",
        description="접수 연도 (4자리 숫자)",
        type="integer"
    ),
    AttributeInfo(
        name="month",
        description="접수 월 (1-12 사이의 숫자)",
        type="integer"
    )
]

# document_content_description도 함께 정의
document_content_description = "고객 상담 내용 및 처리 기록. 가맹점 관련 문의, 불만, 요청사항 등이 포함됨"

In [16]:
from langchain_community.query_constructors.pgvector import PGVectorTranslator

# Your pgvector_translator import
from langchain_core.structured_query import Comparator, Operator
from langchain_classic.retrievers.self_query.pgvector import PGVectorTranslator
from langchain_classic.retrievers import SelfQueryRetriever
from langchain_classic.chains.query_constructor.schema import AttributeInfo
from langchain_classic.retrievers import EnsembleRetriever
from kiwipiepy import Kiwi
from langchain_core.retrievers import BaseRetriever
from typing import List
from langchain_core.documents import Document

#hybrid 추가 library
from typing import List, Any, Optional
from pydantic import ConfigDict

In [17]:
pgvector_translator = PGVectorTranslator()
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

retriever = SelfQueryRetriever.from_llm(
    llm,
    vectorstore,
    document_content_description,
    metadata_field_info,
    structured_query_translator=pgvector_translator,
    verbose=True,
    # chain_kwargs에 allowed_comparators 추가
    chain_kwargs={
        "allowed_comparators": [
            Comparator.EQ,
            Comparator.NE,
            Comparator.GT,
            Comparator.GTE,
            Comparator.LT,
            Comparator.LTE,
            Comparator.IN,
            Comparator.NIN,
            Comparator.LIKE,
        ],
        "allowed_operators": [
            Operator.AND,
            Operator.OR,
            Operator.NOT,
        ]
    }
)

In [18]:
# retriever.invoke("6월동안 10000개 이상으로 누적된 매장에 대한  문의사항")
# 이라고 하면 안나옴
# retriever.invoke("1월에서 2월 사이 투썸플레이스 문의사항 개수는?")
#여전히 안나옴 

In [19]:
# results = vectorstore.similarity_search(
#     "투썸플레이스",
#     k=50000,
#     filter={
#         "$and": [
#             {"month": {"$gte": 1}},
#             {"month": {"$lte": 2}}
#         ]
#     }
# )
# print(f"1-2월 필터: {len(results)}개")


In [20]:

# # 개선된 프롬프트
# filter_prompt = ChatPromptTemplate.from_messages([
#     ("system", """사용자 질문을 분석하여 검색 쿼리와 PGVector 필터를 생성하세요.

# 메타데이터 필드:
# - no, date, category, store_name, summary, type, main_category, sub_category, year, month

# **중요한 필터 규칙:**
# 1. 각 필드는 단일 연산자만 가능: {{"field": {{"$operator": value}}}}
# 2. 범위 조건은 반드시 $and 사용: {{"$and": [{{"month": {{"$gte": 6}}}}, {{"month": {{"$lte": 12}}}}]}}
# 3. 잘못된 예: {{"month": {{"$gte": 6, "$lte": 12}}}} ❌
# 4. 올바른 예: {{"$and": [{{"month": {{"$gte": 6}}}}, {{"month": {{"$lte": 12}}}}]}} 

# **PGVector 필터 형식 (중요!):**
# - 범위 조건: {{"month": {{"$gte": 1, "$lte": 2}}}} (한 필드에 여러 연산자 가능)
# - 단일 조건: {{"year": {{"$eq": 2024}}}}
# - 필터 없음: null

# 연산자: $eq, $ne, $gt, $gte, $lt, $lte, $in, $nin, $like

# {format_instructions}
# 웬만하면 장소는 category로 filtering할 것 

# 예시:
# Q: "1월에서 2월 사이의 투썸플레이스"
# A: {{"category": "투썸플레이스", "filter": {{"$and": [{{"month": {{"$gte": 1}}}}, {{"month": {{"$lte": 2}}}}]}}}}

# Q: "2024년 클레임"
# A: {{"query": "클레임", "filter": {{"year": {{"$eq": 2024}}}}}}

# Q: "투썸플레이스"
# A: {{"category": "투썸플레이스", "filter": null}}

# Q: "처리완료된 상담"
# A: {{"query": "상담", "filter": {{"type": {{"$eq": "처리완료"}}}}}}
# """),
#     ("user", "{question}")
# ])


In [21]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from pydantic import BaseModel, Field
from typing import Optional, Dict, Any

class SearchQuery(BaseModel):
    query: str = Field(description="벡터 검색 키워드")
    filter: Optional[Dict[str, Any]] = Field(default=None, description="PGVector 필터")

parser = JsonOutputParser(pydantic_object=SearchQuery)

# 개선된 프롬프트
filter_prompt = ChatPromptTemplate.from_messages([
    ("system", """사용자 질문을 분석하여 검색 쿼리와 PGVector 필터를 생성하세요.


메타데이터 필드:
- no, date, category, store_name, summary, type, main_category, sub_category, year, month

**중요한 규칙:**
1. 가맹점명(투썸플레이스, 스타벅스 등)은 'category' 필드로 필터링할 것
2. 범위 조건은 반드시 $and 사용: {{"$and": [{{"month": {{"$gte": 1}}}}, {{"month": {{"$lte": 2}}}}]}}
3. 각 필드는 단일 연산자만 가능: {{"field": {{"$operator": value}}}}

연산자: $eq, $ne, $gt, $gte, $lt, $lte, $in, $nin, $like

{format_instructions}

예시:
Q: "1월에서 2월 사이의 투썸플레이스"
A: {{"query": "", "filter": {{"$and": [{{"category": {{"$eq": "투썸플레이스"}}}}, {{"month": {{"$gte": 1}}}}, {{"month": {{"$lte": 2}}}}]}}}}

Q: "2024년 클레임"
A: {{"query": "클레임", "filter": {{"year": {{"$eq": 2024}}}}}}

Q: "투썸플레이스"
A: {{"query": "", "filter": {{"category": {{"$eq": "투썸플레이스"}}}}}}

Q: "투썸플레이스 키오스크 문의"
A: {{"query": "키오스크", "filter": {{"category": {{"$eq": "투썸플레이스"}}}}}}

Q: "처리완료된 상담"
A: {{"query": "상담", "filter": {{"type": {{"$eq": "처리완료"}}}}}}

Q: "1월 투썸플레이스 영수증프린터"
A: {{"query": "영수증프린터", "filter": {{"$and": [{{"category": {{"$eq": "투썸플레이스"}}}}, {{"month": {{"$eq": 1}}}}]}}}}
"""),
    ("user", "{question}")
])

filter_chain = filter_prompt | llm | parser

# 답변 생성 체인
# answer_prompt = ChatPromptTemplate.from_messages([
#     ("system", "검색 결과를 바탕으로 사용자 질문에 간결하고 명확하게 답변하세요."),
#     ("user", """질문: {question}

# 검색 결과:
# {context}

# 위 결과를 바탕으로 답변해주세요.""")
# ])

# answer_prompt = ChatPromptTemplate.from_messages([
#     ("system", """You are a customer service case analysis expert. 
# Based on the retrieved consultation cases, provide your answer in the following format:

# 1. Present **5 cases** that match the query in the following format:
#    - Store Name: 
#    - Franchise Category:
#    - Receipt Date:
#    - Issue Summary:
#    - Detailed Content:
#    - Resolution Result:

# 2. At the end, provide a brief **overall analysis** of all cases

# **IMPORTANT: Respond in Korean, even though these instructions are in English.**
# """),
#     ("user", """Question: {question}

# Search Results:
# {context}

# Please answer according to the format above, in Korean.""")
# ])

prompt = PromptTemplate.from_template(
    """You are an assistant for AS call center worker who needs information for error case - answer tasks. 
Use the following pieces of retrieved context to answer the question. 
If you don't know the answer, just say that you don't know. 
Provide several cases including details and then one summary sentence. 
Answer in Korean.

IMPORTANT: Ignore documents without exact match, Priority is user's query. Strictly follow the query.


#Context: 
{context}

#Question:
{question}

#Answer:"""
)


# answer_chain = answer_prompt | llm
answer_chain = prompt | llm

# 전체 RAG 파이프라인

def rag_pipeline(question: str, k: int = 30):
    # Step 1-2: 자연어 → 필터 변환
    query_info = filter_chain.invoke({
        "question": question,
        "format_instructions": parser.get_format_instructions()
    })
    
    print(f"🔍 검색어: {query_info['query']}")
    print(f"🔧 필터: {query_info['filter']}")
    
    # Step 3: 검색 실행
    #가장 유사도 높은 거 30개를 조회하라 -> 여기서 통계적 수치는 lost 된다. 
    docs = vectorstore.similarity_search(
        query_info['query'],
        k=k,
        filter=query_info['filter']
    )
    
    print(f"📄 검색 결과: {len(docs)}건\n")
    
    if len(docs) == 0:
        return "검색 결과가 없습니다."
    
    # 검색 결과를 컨텍스트로 포맷팅
    context = "\n\n".join([
        f"[{i+1}] 가맹점: {doc.metadata.get('store_name')}\n"
        f"일자: {doc.metadata.get('date')}\n"
        f"대분류: {doc.metadata.get('main_category')}\n"
        f"중분류: {doc.metadata.get('sub_category')}\n"
        f"내용: {doc.page_content[:200]}"
        for i, doc in enumerate(docs[:10])
    ])
    
    # Step 4-5: LLM이 답변 생성
    response = answer_chain.invoke({
        "question": question,
        "context": context
    })
    
    return response.content

# # 사용
# answer = rag_pipeline("1월에서 2월 사이의 투썸플레이스 영수증프린터 문의사항을 5개 보여줘")
# print(f"💬 답변:\n{answer}")

In [22]:
%%time
answer = rag_pipeline("1월에서 2월 사이 투썸플레이스 키오스크 문의사항을 보여줘.")
print(f"💬 답변:\n{answer}")

🔍 검색어: 키오스크
🔧 필터: {'$and': [{'category': {'$eq': '투썸플레이스'}}, {'month': {'$gte': 1}}, {'month': {'$lte': 2}}]}
📄 검색 결과: 30건

💬 답변:
다음은 2025년 1월에서 2월 사이 투썸플레이스 키오스크에 대한 문의사항입니다.

1. **투썸플레이스(수원천천점)**
   - **일자:** 2025-01-20
   - **문제 요약:** 키오스크 소리 출력 안 됨
   - **처리 결과:** 재생디바이스 변경 후 정상
   - **담당자:** 고정현

2. **투썸플레이스(이천온천사거리점)**
   - **일자:** 2025-02-13
   - **문제 요약:** 키오스크 소리 출력 안 됨
   - **처리 결과:** 재생디바이스 변경 후 정상
   - **담당자:** 고정현

3. **투썸플레이스(부천중동역점)**
   - **일자:** 2025-02-16
   - **문제 요약:** 키오스크 소리 출력 안 됨
   - **처리 결과:** 소리 설정 변경 후 정상
   - **담당자:** 이은혜

4. **투썸플레이스(대전죽동점)**
   - **일자:** 2025-02-13
   - **문제 요약:** 키오스크 소리 출력 안 됨
   - **처리 결과:** 재생디바이스 변경 후 정상
   - **담당자:** 고정현

5. **투썸플레이스 서울숲역점**
   - **일자:** 2025-01-16
   - **문제 요약:** 키오스크 2대 소리 출력 안 됨
   - **처리 결과:** 재생디바이스 변경, 볼륨 최대로 설정 후 정상
   - **담당자:** 고정현

6. **투썸플레이스(부산서대신점)**
   - **일자:** 2025-01-17
   - **문제 요약:** 키오스크 음성 소리 안들림
   - **처리 결과:** 소리 설정 변경 후 정상
   - **담당자:** 이은혜

요약하자면, 1월과 2월 사이 투썸플레이스의 키오스크 관련 문의는 주로 소리 출력 문제였으며

In [23]:
%%time
answer = rag_pipeline("투썸 매장에서 POS 프린터 오류 어떻게 처리했어?")
print(f"💬 답변:\n{answer}")

🔍 검색어: POS 프린터 오류
🔧 필터: {'category': {'$eq': '투썸플레이스'}}
📄 검색 결과: 30건

💬 답변:
투썸 매장에서 POS 프린터 오류 처리 사례는 다음과 같습니다:

1. **투썸 박달사거리(2)** (2025-08-10)
   - 문제: POS프린터 출력이상 (흐림, 잘림, 안나옴)
   - 처리 결과: 선 재연결 및 셀프테스트 후 정상 작동.

2. **투썸 대림삼거리(4)** (2025-09-05)
   - 문제: POS프린터 출력이상 (흐림, 잘림, 안나옴)
   - 처리 결과: POST 주방프린터 사용여부 미사용 처리.

3. **투썸플레이스(일산화정(3)점)** (2025-12-24)
   - 문제: POS프린터 출력이상 (흐림, 잘림, 안나옴)
   - 처리 결과: 주방프린터 연결 후 테스트 시 정상 작동.

4. **투썸플레이스(이천온천사거리점)** (2025-11-04)
   - 문제: POS프린터 출력이상 (흐림, 잘림, 안나옴)
   - 처리 결과: 포스 프로그램 외 문서 출력 방법 문의, 어려움 안내.

5. **투썸 북서울꿈의숲(2)** (2025-09-21)
   - 문제: 새로 설치한 주방프린터가 출력 안 됨.
   - 처리 결과: 장치관리자 포트 삭제 후 재부팅, 버츄얼 재설정하여 정상 출력.

요약: 투썸 매장에서는 POS 프린터 오류를 주로 원격으로 점검하고, 연결 상태 확인, 설정 변경, 셀프테스트 등을 통해 문제를 해결했습니다.
CPU times: total: 46.9 ms
Wall time: 14.1 s


In [27]:
%%time
answer = rag_pipeline("투섬 매장에서 포스 프린터 오류 어떻게 처리했어?")
print(f"💬 답변:\n{answer}")

🔍 검색어: 포스 프린터 오류
🔧 필터: {'category': {'$eq': '투썸플레이스'}}
📄 검색 결과: 30건

💬 답변:
투썸 매장에서의 포스 프린터 오류 처리 사례는 다음과 같습니다:

1. **투썸플레이스(이대역점)**  
   - **일자**: 2025-05-26  
   - **문제 요약**: POS프린터 출력이상(흐림, 잘림, 안나옴)  
   - **상세 내용**: 1번 포스 프린터 에러 발생  
   - **처리 결과**: 용지 교체 후 정상 작동  
   - **담당자**: 고정현  

2. **투썸플레이스 청주성화(2)**  
   - **일자**: 2025-08-14  
   - **문제 요약**: POS프린터 출력이상(흐림, 잘림, 안나옴)  
   - **상세 내용**: 포스 프린터 에러 발생 및 출력 안 됨  
   - **처리 결과**: 셀프 테스트 후 정상 작동  
   - **담당자**: 송미현  

3. **투썸플레이스(안양교보생명)**  
   - **일자**: 2025-12-30  
   - **문제 요약**: POS프린터 출력이상(흐림, 잘림, 안나옴)  
   - **상세 내용**: 01포스 프린터 에러 발생 및 영수증 출력 안 됨  
   - **처리 결과**: 셀프 테스트 후 정상 작동  
   - **담당자**: 고정현  

4. **투썸(춘천석사후석로점)**  
   - **일자**: 2025-08-12  
   - **문제 요약**: POS프린터 출력이상(흐림, 잘림, 안나옴)  
   - **상세 내용**: 1번 포스 영수증 프린터 C 에러 발생 및 영수증 출력 안 됨  
   - **처리 결과**: 선재 연결 후 정상 작동  
   - **담당자**: 고정현  

5. **투썸플레이스(광주운암점)**  
   - **일자**: 2025-12-04  
   - **문제 요약**: POS프린터 출력이상(흐림, 잘림, 안나옴)  
   - **상세 내용**: 1포스 프린터 전원 안 켜짐  
   - **

In [24]:
%%time
answer = rag_pipeline("투썸플레이스에서 POS 프린터 오류 어떻게 처리했어?")
print(f"💬 답변:\n{answer}")

🔍 검색어: POS 프린터 오류
🔧 필터: {'category': {'$eq': '투썸플레이스'}}
📄 검색 결과: 30건

💬 답변:
투썸플레이스에서 POS 프린터 오류 처리 사례는 다음과 같습니다:

1. **투썸 박달사거리(2)** (2025-08-10)
   - 문제: POS프린터 출력이상 (흐림, 잘림, 안나옴)
   - 처리 결과: 선 재연결 및 셀프테스트 후 정상 작동.

2. **투썸 대림삼거리(4)** (2025-09-05)
   - 문제: POS프린터 출력이상 (흐림, 잘림, 안나옴)
   - 처리 결과: POST 주방프린터 사용여부 미사용 처리.

3. **투썸플레이스(일산화정(3)점)** (2025-12-24)
   - 문제: POS프린터 출력이상 (흐림, 잘림, 안나옴)
   - 처리 결과: 주방프린터 연결 후 테스트 시 정상 작동.

4. **투썸플레이스(이천온천사거리점)** (2025-11-04)
   - 문제: POS프린터 출력이상 (흐림, 잘림, 안나옴)
   - 처리 결과: 포스 프로그램 외 문서 출력 방법 문의, 어려움 안내.

5. **투썸 북서울꿈의숲(2)** (2025-09-21)
   - 문제: 새로 설치한 주방프린터가 출력 안 됨.
   - 처리 결과: 장치관리자 포트 삭제 후 재부팅, 버츄얼 재설정하여 정상 출력.

요약: 투썸플레이스에서는 POS 프린터 오류를 주로 원격으로 점검하고, 재연결, 설정 변경, 셀프테스트 등을 통해 문제를 해결했습니다.
CPU times: total: 78.1 ms
Wall time: 13.3 s


In [25]:
import tiktoken
from langchain_community.callbacks import get_openai_callback

# 인코더 초기화
encoding = tiktoken.encoding_for_model("gpt-4o-mini")

def count_tokens(text):
    return len(encoding.encode(text))

# 질문
question = "투썸 매장에서 POS 프린터 오류 어떻게 처리했어?"

# 토큰 추적하면서 RAG 실행
with get_openai_callback() as cb:
    response = rag_pipeline(question)
    
    print("=" * 60)
    print("📊 토큰 사용량")
    print("=" * 60)
    print(f"입력 토큰:         {cb.prompt_tokens:,}")
    print(f"출력 토큰:         {cb.completion_tokens:,}")
    print(f"총 토큰:           {cb.total_tokens:,}")
    print(f"비용 ($):          ${cb.total_cost:.6f}")
    print(f"비용 (원):         {cb.total_cost * 1458:.2f}원")
    print()

print("💬 답변:")
print(response)

🔍 검색어: POS 프린터 오류
🔧 필터: {'category': {'$eq': '투썸플레이스'}}
📄 검색 결과: 30건

📊 토큰 사용량
입력 토큰:         2,880
출력 토큰:         434
총 토큰:           3,314
비용 ($):          $0.000548
비용 (원):         0.80원

💬 답변:
투썸 매장에서 POS 프린터 오류 처리 사례는 다음과 같습니다:

1. **투썸 박달사거리(2)** (2025-08-10)
   - 문제: POS프린터 출력이상 (흐림, 잘림, 안나옴)
   - 처리 결과: 선 재연결 및 셀프테스트 후 정상 작동.

2. **투썸 대림삼거리(4)** (2025-09-05)
   - 문제: POS프린터 출력이상 (흐림, 잘림, 안나옴)
   - 처리 결과: POST 주방프린터 사용여부 미사용 처리.

3. **투썸플레이스(일산화정(3)점)** (2025-12-24)
   - 문제: POS프린터 출력이상 (흐림, 잘림, 안나옴)
   - 처리 결과: 주방프린터 연결 후 테스트 시 정상 작동.

4. **투썸플레이스(이천온천사거리점)** (2025-11-04)
   - 문제: POS프린터 출력이상 (흐림, 잘림, 안나옴)
   - 처리 결과: 포스 프로그램이 아닌 다른 문서를 출력하는 방법에 대한 안내 어려움.

5. **투썸플레이스(목동메가박스점)** (2025-08-27)
   - 문제: POS프린터 출력이상 (흐림, 잘림, 안나옴)
   - 처리 결과: 내부 청소 및 용지 교체 후 셀프테스트로 정상 작동.

이와 같이, 투썸 매장에서는 POS 프린터 오류를 주로 원격으로 점검하고, 재연결, 셀프테스트, 설정 변경 등을 통해 문제를 해결하고 있습니다.


In [26]:
monthly_mean = 5840.5  # 한 달 평균 건수
monthly_std = 651.9    # 표준편차
questions_per_case = 1  # 1건당 질문 수
Kwon_total_token_price
cost_per_question = Kwon_total_token_price  # 1 질문당 비용 (원)

# 계산
total_questions_mean = monthly_mean * questions_per_case
total_questions_std = monthly_std * questions_per_case

total_cost_mean = total_questions_mean * cost_per_question
total_cost_std = total_questions_std * cost_per_question

print("=" * 60)
print("💰 한 달 LLM 비용 계산")
print("=" * 60)
print(f"월 평균 케이스:        {monthly_mean:,.1f}건")
print(f"1건당 질문 수:         {questions_per_case}번")
print(f"1질문당 비용:          {cost_per_question}원")
print()
print(f"월 총 질문 수:         {total_questions_mean:,.1f}회")
print(f"                       (±{total_questions_std:,.1f}회)")
print()
print(f"💵 월 평균 비용:       {total_cost_mean:,.0f}원")
print(f"   (±{total_cost_std:,.0f}원)")
print()
print(f"예상 범위:")
print(f"  최소: {total_cost_mean - total_cost_std:,.0f}원")
print(f"  평균: {total_cost_mean:,.0f}원")
print(f"  최대: {total_cost_mean + total_cost_std:,.0f}원")
print()
print(f"💳 연간 예상 비용:     {total_cost_mean * 12:,.0f}원")
print(f"                       ({total_cost_mean * 12 / 10000:.1f}만원)")

NameError: name 'Kwon_total_token_price' is not defined

In [ ]:
# #마찬가지로, 딱 봐도 1월 첫주에서 10건이 넘는데 통계적 질문은 어려움 - 유사도 높은 것 개수만 조절가능하기 때문에 검색결과 사이즈에서 문제가 생긴다. 
# answer = rag_pipeline("1월에서 2월 투썸플레이스의 문의사항이 몇 건이나 들어왔어?")
# print(f"💬 답변:\n{answer}")